In [7]:
from audio_test import record_loop_chunk, VADe, transcribe_w2v2_clean
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [2]:
record_loop_chunk(duration=10, bruit_reduction=True)

Parlez (Ctrl+C pour arrêter).

Arrêt demandé par l'utilisateur (Ctrl+C).


In [5]:
folder = Path("tests")

for audio_path in folder.glob("*.wav"): 
    VADe(audio_path, min_duration_on=1, min_duration_off=2)     

/Users/mohammedlbakali/Desktop/DTY/detection-notes/.venv/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/mohammedlbakali/Desktop/DTY/detection-notes/.venv/lib/python3.12/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec

In [6]:
folder = Path("tmp")

for audio_path in folder.glob("*.wav"): 
    transcribe_w2v2_clean(audio_path)